# Analysis basics

Simple example of an "analysis" using a final state with two leptons. In this example we show how you can read data from a ROOT file containing a TTree, run through the events, and produce some histograms of the data. 

Start by including some stuff that we need: 

In [1]:
#include <iostream>
#include <string>
#include <stdio.h>

In [2]:
%jsroot on

Your first ROOT command is to make a [TChain](https://root.cern.ch/doc/master/classTChain.html), and add the data files you want to analyse to this chain. Your TChain will contain the ROOT files you want to analyse, and each of these files contains a [TTree](https://root.cern.ch/doc/master/classTTree.html), all with the same structure (i.e. same variables/variable names). When doing the analysis you will run through all events in the files you have added to the chain.   

In [3]:
TChain *dataset = new TChain("mini"); 

In [4]:
//dataset->Reset(); // You can reset the chain if you want to "start over". 

In [4]:
dataset->Add("DataSamples/MC/mc_147770.Zee.root");

Define the variables you need in your analysis: 

In [5]:
const int vs = 5; 

Int_t lepton_n = -1, lepton_charge[vs], lepton_type[vs]; 

Float_t lepton_pt[vs], lepton_E[vs], lepton_phi[vs], lepton_eta[vs];

Relate your variables to branches (variables) in the TTree: 

In [6]:
dataset->SetBranchAddress("lep_n",      &lepton_n);
dataset->SetBranchAddress("lep_charge", &lepton_charge);
dataset->SetBranchAddress("lep_type",   &lepton_type);
dataset->SetBranchAddress("lep_pt",     &lepton_pt);
dataset->SetBranchAddress("lep_eta",    &lepton_eta);
dataset->SetBranchAddress("lep_phi",    &lepton_phi);
dataset->SetBranchAddress("lep_E",      &lepton_E);

Define the histograms you want to make! (E.g. $p_T$, $\phi$, $\eta$, energy and/or invariant mass of leptons, missing $E_T$, jet variables etc.) The most commonly used type of histogram in ROOT is [TH1F](https://root.cern.ch/doc/master/classTH1F.html), which is a 1D histogram of float numbers. However, ROOT also has a variety of other histogram options, see [TH1](https://root.cern.ch/doc/master/classTH1.html). 

In [7]:
TH1F *hist_mll = new TH1F("mll", "Invariant mass", 20, 0, 200);

TH1F *hist_E = new TH1F("lep_E", "Lepton energy", 20, 0, 1000);

TH1F *hist_pT = new TH1F("lep_pT", "Lepton pT", 20, 0, 1000);

In [8]:
int nentries, i; 
nentries = (Int_t)dataset->GetEntries(); // Get number of entries (i.e. number of events) in your data set.

fraction_events = 1; // Specify the fraction of events you want to run over. 
events_to_run = nentries*fraction_events;

std::cout << "Total # events = "  << nentries
          << ". Events to run = " << events_to_run
          << " corresponding to " << fraction_events*100
          << "% of total events!" << std::endl;


Total # events = 7500000. Events to run = 7500000 corresponding to 100% of total events!


Make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s to easily handle the kinematics of the leptons: 

In [9]:
TLorentzVector l1; TLorentzVector l2; TLorentzVector dileptons;

It is now time to run through the events in your chain, add the desired cuts. If an events passes all your cuts, you fill your histograms.  

In [10]:
hist_mll->Reset(); hist_E->Reset(); hist_pT->Reset(); // Reset your histograms, in case you have filled them before

for (i = 0; i < nentries; i++)
{
    dataset->GetEntry(i); // We "pull out" the i'th entry in the chain. The variables are now 
                          // available through the names we have given them. 
    
    // Cut #1: At least 2 leptons
    if(lepton_n == 2)
    {
        // Cut #2: Leptons with opposite charge
        if(lepton_charge[0] != lepton_charge[1])
        {
            // Cut #3: Leptons of the same family (2 electrons or 2 muons)
            if(lepton_type[0] == lepton_type[1])
            {
                l1.SetPtEtaPhiE(lepton_pt[0]/1000., lepton_eta[0], lepton_phi[0], lepton_E[0]/1000.);
                l2.SetPtEtaPhiE(lepton_pt[1]/1000., lepton_eta[1], lepton_phi[1], lepton_E[1]/1000.);
                // Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
                // to get GeV, which is a more practical unit. 
                
                dileptons = l1 + l2;
                hist_mll->Fill(dileptons.M());
                hist_E->Fill(l1.E());
                hist_E->Fill(l2.E());
                hist_pT->Fill(l1.Pt()); 
                hist_pT->Fill(l2.Pt()); 
            }
        }
    }
}

When you have filled your histograms you can visualize them by making a [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html), and then draw the histogram and the canvas. Before drawing the histograms you might also want to decorate the histogram a little bit, e.g. adding axis labels, colours etc., and the TH1F class contains a lot of functionality for doing such things. Notice however that most of these functions are inherited from other classes, so you might have to "click around" a little bit in the documentation to find what you are looking for. 

In [11]:
TCanvas *c = new TCanvas("c", "c", 10, 10, 700, 700);

In [12]:
hist_mll->GetYaxis()->SetTitle("# events"); 
hist_mll->GetXaxis()->SetTitle("m_{ll} (GeV)");
hist_mll->GetXaxis()->SetTitleOffset(1.3);
hist_mll->SetFillColor(kBlue);

In [13]:
hist_mll->Draw(); 
c->Draw();

To draw the next histogram you first draw the histogram, and then draw the canvas again. (If needed you can of course also make a new canvas.) You can also add a legend by using the 
[TLegend](https://root.cern.ch/doc/master/classTLegend.html) class, as shown below.  

In [14]:
hist_E->GetYaxis()->SetTitle("# events"); 
hist_E->GetXaxis()->SetTitle("Energy (GeV)");
hist_E->GetXaxis()->SetTitleOffset(1.3);
hist_E->SetFillColor(kBlue);

In [15]:
gStyle->SetLegendBorderSize(0); 

In [16]:
TLegend *leg = new TLegend(0.65, 0.80, 0.85, 0.85);

In [17]:
hist_E->SetStats(0); // Remove statistics box 
gStyle->SetLegendBorderSize(0); // Remove (default) border around legend 
leg->Clear();
leg->AddEntry(hist_E, "Zee", "f");  // Add your histogram to the legend

In [18]:
hist_E->Draw(); 
leg->Draw(); // Draw the legend 
c->Draw();